# Import 

In [30]:

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from sklearn.utils import resample

import warnings
warnings.filterwarnings('ignore')


In [2]:
try:
    # Try loading the file from your laptop path
    df1 = pd.read_csv('C:/Users/Eugene/Documents/GitHub/TSDN-BoyWithLuv/Source/Data/train.csv')
except FileNotFoundError:
    # If the file is not found, try loading from the PC path
    df1 = pd.read_csv('C:/Users/user/OneDrive/Documents/GitHub/TSDN-BoyWithLuv/Source/Data/train.csv')

In [3]:
df1

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,318434,6,a,6,X,3,radiotherapy,Q,F,4.0,86499,23.0,Emergency,Moderate,3,41-50,4144.0,11-20
318434,318435,24,a,1,X,2,anesthesia,Q,E,4.0,325,8.0,Urgent,Moderate,4,81-90,6699.0,31-40
318435,318436,7,a,4,X,3,gynecology,R,F,4.0,125235,10.0,Emergency,Minor,3,71-80,4235.0,11-20
318436,318437,11,b,2,Y,3,anesthesia,Q,D,3.0,91081,8.0,Trauma,Minor,5,11-20,3761.0,11-20


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   Hospital_type_code                 318438 non-null  object 
 3   City_Code_Hospital                 318438 non-null  int64  
 4   Hospital_region_code               318438 non-null  object 
 5   Available Extra Rooms in Hospital  318438 non-null  int64  
 6   Department                         318438 non-null  object 
 7   Ward_Type                          318438 non-null  object 
 8   Ward_Facility_Code                 318438 non-null  object 
 9   Bed Grade                          318325 non-null  float64
 10  patientid                          318438 non-null  int64  
 11  City_Code_Patient                  3139

In [5]:
try:
    # Try loading the file from your laptop path
    df2 = pd.read_csv('C:/Users/Eugene/Documents/GitHub/TSDN-BoyWithLuv/Source/Data/test.csv')
except FileNotFoundError:
    # If the file is not found, try loading from the PC path
    df2 = pd.read_csv('C:/Users/user/OneDrive/Documents/GitHub/TSDN-BoyWithLuv/Source/Data/test.csv')

In [6]:
df2

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit
0,318439,21,c,3,Z,3,gynecology,S,A,2.0,17006,2.0,Emergency,Moderate,2,71-80,3095.0
1,318440,29,a,4,X,2,gynecology,S,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4018.0
2,318441,26,b,2,Y,3,gynecology,Q,D,4.0,17006,2.0,Emergency,Moderate,3,71-80,4492.0
3,318442,6,a,6,X,3,gynecology,Q,F,2.0,17006,2.0,Trauma,Moderate,3,71-80,4173.0
4,318443,28,b,11,X,2,gynecology,R,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4161.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137052,455491,11,b,2,Y,4,anesthesia,Q,D,3.0,41160,3.0,Emergency,Minor,4,41-50,6313.0
137053,455492,25,e,1,X,2,radiotherapy,R,E,4.0,30985,7.0,Emergency,Moderate,2,0-10,3510.0
137054,455493,30,c,3,Z,2,anesthesia,R,A,4.0,81811,12.0,Urgent,Minor,2,0-10,7190.0
137055,455494,5,a,1,X,2,anesthesia,R,E,4.0,57021,10.0,Trauma,Minor,2,41-50,5435.0


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137057 entries, 0 to 137056
Data columns (total 17 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            137057 non-null  int64  
 1   Hospital_code                      137057 non-null  int64  
 2   Hospital_type_code                 137057 non-null  object 
 3   City_Code_Hospital                 137057 non-null  int64  
 4   Hospital_region_code               137057 non-null  object 
 5   Available Extra Rooms in Hospital  137057 non-null  int64  
 6   Department                         137057 non-null  object 
 7   Ward_Type                          137057 non-null  object 
 8   Ward_Facility_Code                 137057 non-null  object 
 9   Bed Grade                          137022 non-null  float64
 10  patientid                          137057 non-null  int64  
 11  City_Code_Patient                  1349

# Preprocessing

## 1

**Hospital Type Code to Assigned Hospital Type**

| **Hospital Type Code** | **Assigned Hospital Type**                    | **Justification**                                                                                                                |
|------------------------|-----------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------|
| **a**                  | **Community Hospital**                        | Community hospitals are small, localized healthcare facilities that primarily serve residents in rural or suburban areas. They provide primary care, emergency services, and general medical treatment to the local population. The 'a' code is mapped here because it represents small, often independent healthcare entities that meet these needs. |
| **b**                  | **General Medical & Surgical Hospital**      |  General medical and surgical hospitals have a broader scope, treating a wide range of medical conditions and performing surgeries. These hospitals offer in-patient and out-patient services and are typically found in urban or metropolitan areas. The 'b' code maps to this category because it reflects general hospital services, including surgical procedures. |
| **c**                  | **Specialty Hospital**                        |  Specialty hospitals focus on specific medical fields like cardiology, oncology, orthopedics, etc. They tend to have advanced equipment and specialized medical teams. The 'c' code is used for hospitals specializing in particular medical services, as it suggests a specialized care unit designed for in-depth treatment in specific areas of healthcare. |
| **d**                  | **Public Hospital**                           |  Public hospitals are typically government-funded institutions that provide comprehensive healthcare services, including emergency care and specialized medical treatment. They are designed to serve the broader public, often with a focus on offering affordable care to a wide range of patients. The 'd' code is used to designate hospitals that are publicly operated and provide services to a large community. |
| **e**                  | **Clinic**                                    |  Clinics are smaller, outpatient-focused healthcare facilities that provide general medical care for minor conditions, diagnostic services, and follow-up care. Clinics do not typically offer emergency care or extensive inpatient services. The 'e' code refers to these smaller, outpatient-based care centers. |
| **f**                  | **Acute Hospital**                            |  Acute hospitals provide care for patients with severe, life-threatening conditions that require immediate attention. These hospitals are equipped with emergency rooms, intensive care units (ICUs), and other specialized units to provide urgent medical intervention. The 'f' code corresponds to these specialized care facilities designed for acute, short-term medical treatment. |
| **g**                  | **Teaching Hospital**                         |  Teaching hospitals are large medical centers affiliated with academic institutions. They provide comprehensive medical services and serve as training grounds for medical professionals. These hospitals also perform complex surgeries and research. The 'g' code reflects hospitals that not only provide patient care but also serve as academic centers for future healthcare professionals. |

**Ward Type to Assigned Hospital Ward Type**

| **Ward Type** | **Assigned Hospital Ward Type**               | **Justification**                                                                                                             |
|---------------|-----------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------|
| **P**         | **Obstetrics & Gynecology Ward**              | The "P" ward type corresponds to wards specifically designed for obstetrics and gynecology, as these wards deal with conditions related to pregnancy, childbirth, and women's health. This ward type is mapped to Obstetrics & Gynecology Ward because the primary focus is maternal and reproductive health. |
| **Q**         | **General Medical Ward**                      | The "Q" ward type is for general medical care, focusing on treating a variety of medical conditions that are less severe and don’t require specialized care. This ward type is typically used for patients who need standard medical care but are not critical. Thus, it’s classified under General Medical Ward. |
| **R**         | **Intermediate Care Ward**                    | The "R" ward type is designed for patients who require closer monitoring than general medical care but do not need full intensive care. This includes patients recovering from surgery or medical procedures who need constant observation. Intermediate care is often given to patients in transitional states. |
| **S**         | **Specialty Care Ward**                       | The "S" ward type is for patients requiring specialized care in specific fields, such as cardiology, neurology, or other advanced treatments. These wards are tailored to meet the needs of patients with complex conditions requiring specialized medical attention. |
| **T**         | **General Ward**                              | The "T" ward type is used for patients with general medical needs who do not require specialized care or intensive monitoring. This ward type focuses on providing routine medical attention and general healthcare services. Therefore, it is mapped to the General Ward, which handles standard, less critical medical conditions. |
| **U**         | **Day Surgery Unit**                          | The "U" ward type is for patients undergoing surgeries that do not require an overnight stay, typically classified as outpatient procedures. This ward is equipped to handle minor surgeries, diagnostic procedures, and post-surgery recovery in a short-term, same-day discharge setup. Thus, it is mapped to the Day Surgery Unit. |

In [8]:
hospital_type_mapping = {
    'a': 'Community Hospital',
    'b': 'General Medical & Surgical Hospital',
    'c': 'Specialty Hospital',
    'd': 'Public Hospital',
    'e': 'Clinic',
    'f': 'Acute Hospital',
    'g': 'Teaching Hospital'
}

ward_type_mapping = {
    'P': 'Obstetrics & Gynecology Ward',
    'Q': 'General Medical Ward',
    'R': 'Intermediate Care Ward',
    'S': 'Specialty Care Ward',
    'T': 'General Ward',
    'U': 'Day Surgery Unit'
}

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   Hospital_type_code                 318438 non-null  object 
 3   City_Code_Hospital                 318438 non-null  int64  
 4   Hospital_region_code               318438 non-null  object 
 5   Available Extra Rooms in Hospital  318438 non-null  int64  
 6   Department                         318438 non-null  object 
 7   Ward_Type                          318438 non-null  object 
 8   Ward_Facility_Code                 318438 non-null  object 
 9   Bed Grade                          318325 non-null  float64
 10  patientid                          318438 non-null  int64  
 11  City_Code_Patient                  3139

In [10]:
df1['Hospital Type'] = df1['Hospital_type_code'].map(hospital_type_mapping)
df1['Ward Type'] = df1['Ward_Type'].map(ward_type_mapping)

In [11]:
df1 = df1.drop(columns = ['Hospital_type_code','Ward_Type'])

In [12]:
df1

,case_id,Hospital_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay,Hospital Type,Ward Type
0,1,8,3,Z,3,radiotherapy,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10,Specialty Hospital,Intermediate Care Ward
1,2,2,5,Z,2,radiotherapy,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50,Specialty Hospital,Specialty Care Ward
2,3,10,1,X,2,anesthesia,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40,Clinic,Specialty Care Ward
3,4,26,2,Y,2,radiotherapy,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50,General Medical & Surgical Hospital,Intermediate Care Ward
4,5,26,2,Y,2,radiotherapy,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50,General Medical & Surgical Hospital,Specialty Care Ward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318433,318434,6,6,X,3,radiotherapy,F,4.0,86499,23.0,Emergency,Moderate,3,41-50,4144.0,11-20,Community Hospital,General Medical Ward
318434,318435,24,1,X,2,anesthesia,E,4.0,325,8.0,Urgent,Moderate,4,81-90,6699.0,31-40,Community Hospital,General Medical Ward
318435,318436,7,4,X,3,gynecology,F,4.0,125235,10.0,Emergency,Minor,3,71-80,4235.0,11-20,Community Hospital,Intermediate Care Ward
318436,318437,11,2,Y,3,anesthesia,D,3.0,91081,8.0,Trauma,Minor,5,11-20,3761.0,11-20,General Medical & Surgical Hospital,General Medical Ward


In [13]:
df2['Hospital Type'] = df2['Hospital_type_code'].map(hospital_type_mapping)
df2['Ward Type'] = df2['Ward_Type'].map(ward_type_mapping)

In [14]:
df2 = df2.drop(columns = ['Hospital_type_code','Ward_Type'])

In [15]:
df2

,case_id,Hospital_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Hospital Type,Ward Type
0,318439,21,3,Z,3,gynecology,A,2.0,17006,2.0,Emergency,Moderate,2,71-80,3095.0,Specialty Hospital,Specialty Care Ward
1,318440,29,4,X,2,gynecology,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4018.0,Community Hospital,Specialty Care Ward
2,318441,26,2,Y,3,gynecology,D,4.0,17006,2.0,Emergency,Moderate,3,71-80,4492.0,General Medical & Surgical Hospital,General Medical Ward
3,318442,6,6,X,3,gynecology,F,2.0,17006,2.0,Trauma,Moderate,3,71-80,4173.0,Community Hospital,General Medical Ward
4,318443,28,11,X,2,gynecology,F,2.0,17006,2.0,Trauma,Moderate,4,71-80,4161.0,General Medical & Surgical Hospital,Intermediate Care Ward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137052,455491,11,2,Y,4,anesthesia,D,3.0,41160,3.0,Emergency,Minor,4,41-50,6313.0,General Medical & Surgical Hospital,General Medical Ward
137053,455492,25,1,X,2,radiotherapy,E,4.0,30985,7.0,Emergency,Moderate,2,0-10,3510.0,Clinic,Intermediate Care Ward
137054,455493,30,3,Z,2,anesthesia,A,4.0,81811,12.0,Urgent,Minor,2,0-10,7190.0,Specialty Hospital,Intermediate Care Ward
137055,455494,5,1,X,2,anesthesia,E,4.0,57021,10.0,Trauma,Minor,2,41-50,5435.0,Community Hospital,Intermediate Care Ward


## 2

In [16]:
df1_droped = df1.drop(columns=['Stay'])

In [17]:
df1_droped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 17 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   City_Code_Hospital                 318438 non-null  int64  
 3   Hospital_region_code               318438 non-null  object 
 4   Available Extra Rooms in Hospital  318438 non-null  int64  
 5   Department                         318438 non-null  object 
 6   Ward_Facility_Code                 318438 non-null  object 
 7   Bed Grade                          318325 non-null  float64
 8   patientid                          318438 non-null  int64  
 9   City_Code_Patient                  313906 non-null  float64
 10  Type of Admission                  318438 non-null  object 
 11  Severity of Illness                3184

In [18]:
df = pd.concat([df1_droped, df2], ignore_index=True)
df

,case_id,Hospital_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Hospital Type,Ward Type
0,1,8,3,Z,3,radiotherapy,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,Specialty Hospital,Intermediate Care Ward
1,2,2,5,Z,2,radiotherapy,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,Specialty Hospital,Specialty Care Ward
2,3,10,1,X,2,anesthesia,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,Clinic,Specialty Care Ward
3,4,26,2,Y,2,radiotherapy,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,General Medical & Surgical Hospital,Intermediate Care Ward
4,5,26,2,Y,2,radiotherapy,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,General Medical & Surgical Hospital,Specialty Care Ward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455490,455491,11,2,Y,4,anesthesia,D,3.0,41160,3.0,Emergency,Minor,4,41-50,6313.0,General Medical & Surgical Hospital,General Medical Ward
455491,455492,25,1,X,2,radiotherapy,E,4.0,30985,7.0,Emergency,Moderate,2,0-10,3510.0,Clinic,Intermediate Care Ward
455492,455493,30,3,Z,2,anesthesia,A,4.0,81811,12.0,Urgent,Minor,2,0-10,7190.0,Specialty Hospital,Intermediate Care Ward
455493,455494,5,1,X,2,anesthesia,E,4.0,57021,10.0,Trauma,Minor,2,41-50,5435.0,Community Hospital,Intermediate Care Ward


# Inital EDA for Further Preprocessing (NO RE-RUN) 

## EDA Hospital Type

In [21]:
df['Hospital_type_code'].unique()

array(['c', 'e', 'b', 'a', 'f', 'd', 'g'], dtype=object)

In [23]:
# Group by Hospital_type_code and Department, then count occurrences
department_counts = df.groupby(['Hospital_type_code', 'Department']).size().unstack(fill_value=0)
department_counts 

Department,TB & Chest disease,anesthesia,gynecology,radiotherapy,surgery
Hospital_type_code,,,,,
a,5383,20603,158645,19132,967
b,3953,10112,75088,9437,294
c,1850,5423,53127,5589,158
d,815,1781,24013,2367,72
e,944,2995,28704,2663,122
f,578,1023,12243,1365,43
g,228,421,4868,480,9


In [24]:
# Group by Hospital_type_code and Severity of Illness, then count occurrences
severity_counts = df.groupby(['Hospital_type_code', 'Severity of Illness']).size().unstack(fill_value=0)
severity_counts

Severity of Illness,Extreme,Minor,Moderate
Hospital_type_code,,,
a,36082,55227,113421
b,17809,26389,54686
c,11851,18052,36244
d,5066,7732,16250
e,6054,9996,19378
f,3034,4007,8211
g,1299,1332,3375


In [27]:
admission_stats = df.groupby('Hospital_type_code')['Admission_Deposit'].describe()
admission_stats

,count,mean,std,min,25%,50%,75%,max
Hospital_type_code,,,,,,,,
a,204730.0,4819.913252,1065.912436,1800.0,4130.00,4686.0,5350.0,11920.0
b,98884.0,4884.064540,1114.995787,1800.0,4169.00,4729.0,5435.0,11293.0
c,66147.0,5100.102650,1111.836681,1801.0,4398.00,4944.0,5609.0,10842.0
d,29048.0,4852.723182,1128.187161,1823.0,4138.00,4694.0,5368.0,10399.0
e,35428.0,4711.825392,985.305395,1809.0,4106.75,4620.0,5159.0,10670.0
f,15252.0,5050.958497,1019.431755,1849.0,4396.00,4918.0,5588.0,10662.0
g,6006.0,4932.395937,1020.260483,1946.0,4293.00,4805.0,5417.0,9873.0


In [25]:
# Get the percentage of each Department within each Hospital_type_code
department_percentage = department_counts.div(department_counts.sum(axis=1), axis=0) * 100
department_percentage

Department,TB & Chest disease,anesthesia,gynecology,radiotherapy,surgery
Hospital_type_code,,,,,
a,2.629317,10.063498,77.489865,9.344991,0.472329
b,3.997613,10.226124,75.935440,9.543506,0.297318
c,2.796801,8.198407,80.316568,8.449363,0.238862
d,2.805701,6.131231,82.666621,8.148582,0.247866
e,2.664559,8.453765,81.020662,7.516653,0.344360
f,3.789667,6.707317,80.271440,8.949646,0.281930
g,3.796204,7.009657,81.052281,7.992008,0.149850


In [26]:
# Get the percentage of each Severity of Illness within each Hospital_type_code
severity_percentage = severity_counts.div(severity_counts.sum(axis=1), axis=0) * 100
severity_percentage

Severity of Illness,Extreme,Minor,Moderate
Hospital_type_code,,,
a,17.624188,26.975529,55.400283
b,18.009992,26.686825,55.303184
c,17.916156,27.290731,54.793112
d,17.440099,26.618012,55.941889
e,17.088179,28.214971,54.696850
f,19.892473,26.271964,53.835563
g,21.628372,22.177822,56.193806


1. **Type `a`**: **Community Hospital**  
   - **Reason**: Type `a` has a moderate average deposit, indicating accessibility for general populations with a range of services. 

2. **Type `b`**: **General Medical & Surgical Hospital**  
   - **Reason**: With slightly higher deposits than `a`, Type `b` likely represents a facility that provides broad medical and surgical services.

3. **Type `c`**: **Specialty Hospital**  
   - **Reason**: The highest mean deposit suggests that Type `c` may focus on specialized care, which is often associated with higher costs.

4. **Type `d`**: **Public Hospital**  
   - **Reason**: Type `d` has similar deposit levels to `a`, but with lower values compared to private or specialty care, making it a suitable candidate for a public, accessible facility.

5. **Type `e`**: **Clinic**  
   - **Reason**: With the lowest mean deposit, Type `e` fits a clinic profile, generally focused on outpatient services and low-cost care.

6. **Type `f`**: **Acute Hospital**  
   - **Reason**: Type `f` has a high deposit level, which aligns with facilities handling acute, intensive treatments and potentially emergency care.

7. **Type `g`**: **Teaching Hospital**  
   - **Reason**: Type `g` has moderately high deposits, and teaching hospitals often offer a range of services while also supporting medical education and training.

| Hospital Type Code | Assigned Hospital Type            |
|--------------------|-----------------------------------|
| a                  | Community Hospital                |
| b                  | General Medical & Surgical Hospital |
| c                  | Specialty Hospital                |
| d                  | Public Hospital                   |
| e                  | Clinic                            |
| f                  | Acute Hospital                    |
| g                  | Teaching Hospital                 |

## EDA Ward Type

In [29]:
df['Ward_Type'].unique()

array(['R', 'S', 'Q', 'P', 'T', 'U'], dtype=object)

In [30]:
df['Department'].unique()

array(['radiotherapy', 'anesthesia', 'gynecology', 'TB & Chest disease',
       'surgery'], dtype=object)

In [32]:
# Grouping by Ward_Type to get summary statistics
ward_stats = df.groupby('Ward_Type').agg({
    'Severity of Illness': lambda x: x.value_counts().idxmax(),  # Mode of Severity for each Ward_Type
    'Available Extra Rooms in Hospital': 'mean',                 # Average extra rooms available
    'Admission_Deposit': ['mean', 'median', 'min', 'max']        # Summary stats for deposit
}).rename(columns={'<lambda_0>': 'Most Common Severity'})

ward_stats

Severity of Illness Available Extra Rooms in Hospital  \
                     <lambda>                              mean   
Ward_Type                                                         
P                    Moderate                          5.272260   
Q                    Moderate                          3.728707   
R                    Moderate                          3.072554   
S                    Moderate                          2.546804   
T                    Moderate                          2.667135   
U                       Minor                          3.250000   

          Admission_Deposit                           
                       mean  median     min      max  
Ward_Type                                             
P               4720.878039  4540.0  1801.0  10553.0  
Q               4785.218789  4637.0  1802.0  11008.0  
R               4937.827997  4805.0  1800.0  11293.0  
S               4918.706961  4778.0  1806.0  11920.0  
T               4649.459916  4567.0  1934.0   9994.0  
U               4686.750000  4501.5  3420.0   6909.0

In [36]:
# Group by Ward_Type and Department to see department distribution across wards
ward_department_counts = df.groupby(['Ward_Type', 'Department']).size().unstack(fill_value=0)

ward_department_counts

Department,TB & Chest disease,anesthesia,gynecology,radiotherapy,surgery
Ward_Type,,,,,
P,257,511,5771,631,29
Q,4136,12842,121823,12702,543
R,5501,18121,142633,16108,576
S,3790,10702,84768,11393,513
T,67,182,1681,199,4
U,0,0,12,0,0


In [33]:
# Convert counts to percentages for easier analysis of department dominance
ward_department_percentage = ward_department_counts.div(ward_department_counts.sum(axis=1), axis=0) * 100

ward_department_percentage

Department,TB & Chest disease,anesthesia,gynecology,radiotherapy,surgery
Ward_Type,,,,,
P,3.569940,7.098208,80.163912,8.765106,0.402834
Q,2.720229,8.446128,80.122463,8.354051,0.357129
R,3.007013,9.905488,77.967519,8.805121,0.314859
S,3.409316,9.627044,76.253531,10.248637,0.461472
T,3.141116,8.532583,78.809189,9.329583,0.187529
U,0.000000,0.000000,100.000000,0.000000,0.000000


In [34]:
# Cross-tabulate Severity of Illness with Ward_Type for clearer distribution
severity_ward_counts = pd.crosstab(df['Ward_Type'], df['Severity of Illness'], normalize='index') * 100
severity_ward_counts

Severity of Illness,Extreme,Minor,Moderate
Ward_Type,,,
P,22.905959,25.281289,51.812752
Q,18.374045,26.127619,55.498336
R,16.744926,28.008790,55.246284
S,18.511955,26.410053,55.077991
T,18.471636,27.473043,54.055321
U,25.000000,41.666667,33.333333


In [35]:
# Display admission deposit statistics grouped by Ward_Type
admission_deposit_stats = df.groupby('Ward_Type')['Admission_Deposit'].describe()
admission_deposit_stats

,count,mean,std,min,25%,50%,75%,max
Ward_Type,,,,,,,,
P,7199.0,4720.878039,1106.020715,1801.0,4018.0,4540.0,5232.0,10553.0
Q,152046.0,4785.218789,1059.013589,1802.0,4110.0,4637.0,5290.0,11008.0
R,182939.0,4937.827997,1080.738532,1800.0,4262.0,4805.0,5459.0,11293.0
S,111166.0,4918.706961,1115.536079,1806.0,4190.0,4778.0,5482.0,11920.0
T,2133.0,4649.459916,1066.193256,1934.0,3962.0,4567.0,5142.0,9994.0
U,12.0,4686.750000,870.848293,3420.0,4261.5,4501.5,5066.0,6909.0


### **1. Ward_Type P: Obstetrics & Gynecology Ward**
#### **Justification:**
- **Department Distribution**: The **majority** of the patients are in the **gynecology** department (about 80%), followed by smaller percentages in **radiotherapy** and **TB & Chest Disease**. This indicates that **gynecology** is the most prominent specialty in this ward.
- **Admission Deposit**: The average admission deposit for Ward P is moderate (mean = 4720), which fits the typical profile for a **specialized ward** like Obstetrics & Gynecology, where procedures or treatments tend to be specialized but not as high-cost as emergency or intensive care.
- **Severity of Illness**: The severity of illness is predominantly **moderate**, indicating that patients in this ward require **routine care** for conditions such as **pregnancy-related issues**, **gynecological surgeries**, or **post-operative care**, which is typical for Obstetrics & Gynecology wards.

#### **Conclusion**: Ward P is most likely a **gynecology-focused ward** that is specialized in **Obstetrics & Gynecology care** but may also handle non-acute cases from other departments like TB or radiotherapy.

---

### **2. Ward_Type Q: General Medical Ward**

#### **Justification:**
- **Department Distribution**: Ward Q has a more **even distribution** of departments, with substantial numbers of **anesthesia** (about 8%), **gynecology** (around 80%), and **radiotherapy**. The balance of multiple specialties suggests this ward is more **generalized** rather than specialized.
- **Admission Deposit**: The admission deposit is moderate (mean = 4785), which is typical of **general medical wards**, where patients might not need the extensive, expensive resources of specialized wards like intensive care or surgery, but still require ongoing care.
- **Severity of Illness**: The **moderate** severity of illness across the board indicates this ward is **not highly specialized** but can handle a range of patients with moderate conditions, typical of **general medical wards**.

#### **Conclusion**: Ward Q is likely a **general medical ward** that caters to a wide range of conditions across specialties like gynecology, radiotherapy, and anesthesia.

---

### **3. Ward_Type R: Specialty Care Ward / Intermediate Care Ward**

#### **Justification:**
- **Department Distribution**: Similar to Ward Q, Ward R has a **high percentage of gynecology patients** (around 78%) but with significant contributions from **anesthesia** (10%) and **radiotherapy** (8%). This indicates a **specialized care** focus, with a mix of **surgical**, **oncological**, and **gynecological care**.
- **Admission Deposit**: The average admission deposit for Ward R is **slightly higher** (mean = 4937), which indicates that this ward handles patients requiring more **complex or specialized** treatments (e.g., surgeries, post-operative care).
- **Severity of Illness**: The **moderate severity** of illness suggests this ward manages patients who need **specialized intervention** but are not in critical or life-threatening condition. This fits the profile of an **Intermediate Care Ward** or a **Specialty Care Ward**, where patients need more focused care than a general ward but do not require the intensive care found in emergency wards.

#### **Conclusion**: Ward R appears to be a **specialized ward**, possibly focused on **post-operative care**, **gynecology**, and **oncology** treatments.

---

### **4. Ward_Type S: Specialty Care Ward**

#### **Justification:**
- **Department Distribution**: The distribution here also shows a **high proportion of gynecology patients** (76%) but also significant numbers in **radiotherapy** (10%) and **anesthesia** (9%). This suggests a **specialized care focus**, possibly treating complex gynecological or cancer-related cases.
- **Admission Deposit**: Ward S has a **slightly higher** average admission deposit (4918), indicating that it handles patients who are likely undergoing **specialized procedures**, possibly in **radiotherapy** or more **invasive gynecological treatments**.
- **Severity of Illness**: Like Ward R, the severity of illness is predominantly **moderate**, but with an even mix of **extreme** and **minor** cases, suggesting a more **specialized care unit** where patients are being treated for more complex or prolonged conditions but are not critically ill.

#### **Conclusion**: Ward S is likely a **specialty care ward**, perhaps focused on **oncology** (radiotherapy) and **gynecological surgeries**.

---

### **5. Ward_Type T: General Ward**

#### **Justification:**
- **Department Distribution**: Ward T has a significant number of **gynecology patients** (78%) but much fewer in the other departments. This indicates that **gynecology** is the primary specialty in this ward, but the care provided is likely to be **routine** and less specialized.
- **Admission Deposit**: The average admission deposit is lower (mean = 4649), which indicates that patients in this ward are **likely receiving standard care** for non-complex conditions, typical of a **general ward**.
- **Severity of Illness**: The severity of illness is **mostly moderate**, which further supports the idea that this ward handles **non-critical cases** with **standard care** needs.

#### **Conclusion**: Ward T is best classified as a **general ward**, which handles patients needing **routine gynecological care** and standard treatments.

---

### **6. Ward_Type U: Day Surgery Unit / Outpatient Ward**

#### **Justification:**
- **Department Distribution**: Ward U has **only gynecology patients** (100%), but the number is very low (12 cases). This suggests it handles **minor procedures** or **short-term care** related to **gynecology**, possibly outpatient or **day surgeries**.
- **Admission Deposit**: The average admission deposit is relatively low (mean = 4686), which is consistent with **day surgery** or **outpatient care**, where patients are not hospitalized for extended periods.
- **Severity of Illness**: The severity of illness is predominantly **minor** (41.67%), further supporting the idea that this ward handles **non-acute, low-severity cases**, which fits the profile of a **day surgery** unit or an **outpatient care facility**.

#### **Conclusion**: Ward U is best classified as a **day surgery unit** or **outpatient ward**, where patients receive **minor gynecological procedures** and do not require extended stays.

---

### Summary of 1-to-1 Assignments:
- **P**: Obstetrics & Gynecology Ward
- **Q**: General Medical Ward
- **R**: Specialty Care Ward / Intermediate Care Ward
- **S**: Specialty Care Ward
- **T**: General Ward
- **U**: Day Surgery Unit / Outpatient Ward

| Ward Type | Assigned Hospital Ward Type            |
|--------------------|-----------------------------------|
| P                  | Obstetrics & Gynecology Ward                |
| Q                  | General Medical Ward |
| R                  | Intermediate Care Ward                |
| S                  | Specialty Care Ward                   |
| T                  | General Ward                            |
| U                  | Day Surgery Unit                    |


# Prediction

## Stay Prediction

In [19]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 18 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   case_id                            318438 non-null  int64  
 1   Hospital_code                      318438 non-null  int64  
 2   City_Code_Hospital                 318438 non-null  int64  
 3   Hospital_region_code               318438 non-null  object 
 4   Available Extra Rooms in Hospital  318438 non-null  int64  
 5   Department                         318438 non-null  object 
 6   Ward_Facility_Code                 318438 non-null  object 
 7   Bed Grade                          318325 non-null  float64
 8   patientid                          318438 non-null  int64  
 9   City_Code_Patient                  313906 non-null  float64
 10  Type of Admission                  318438 non-null  object 
 11  Severity of Illness                3184

In [20]:
df_stay = df1[['Stay',
              'Age',
              'Severity of Illness',
              'Type of Admission',
              'Department',
              'Hospital Type'
]]

In [21]:
df_stay['Stay'].value_counts()

Stay
21-30                 87491
11-20                 78139
31-40                 55159
51-60                 35018
0-10                  23604
41-50                 11743
71-80                 10254
More than 100 Days     6683
81-90                  4838
91-100                 2765
61-70                  2744
Name: count, dtype: int64

In [22]:
df_stay['Age'].value_counts()

Age
41-50     63749
31-40     63639
51-60     48514
21-30     40843
71-80     35792
61-70     33687
11-20     16768
81-90      7890
0-10       6254
91-100     1302
Name: count, dtype: int64

In [23]:
df_stay['Severity of Illness'].value_counts()

Severity of Illness
Moderate    175843
Minor        85872
Extreme      56723
Name: count, dtype: int64

In [24]:
df_stay['Type of Admission'].value_counts()

Type of Admission
Trauma       152261
Emergency    117676
Urgent        48501
Name: count, dtype: int64

In [25]:
df_stay['Department'].value_counts()

Department
gynecology            249486
anesthesia             29649
radiotherapy           28516
TB & Chest disease      9586
surgery                 1201
Name: count, dtype: int64

In [26]:
df_stay['Hospital Type'].value_counts()

Hospital Type
Community Hospital                     143425
General Medical & Surgical Hospital     68946
Specialty Hospital                      45928
Clinic                                  24770
Public Hospital                         20389
Acute Hospital                          10703
Teaching Hospital                        4277
Name: count, dtype: int64

In [22]:
stay_mapping_1 = {
    '0-10': 'Brief Stay',                 #23604
    '11-20': 'Short-term Stay',                #78139
    '21-30': 'Standard Stay',                #87491
    '31-40': 'Moderate Stay',                #55159
    '41-50': 'Prolonged Stay',                #11743
    '51-60': 'Prolonged Stay',                #35018
    '61-70': 'Extended Stay',                #2744
    '71-80': 'Extended Stay',                #10254
    '81-90': 'Extended Stay',                #4838
    '91-100': 'Extended Stay',               #2765
    'More than 100 Days': 'Extended Stay'    #6683
}

age_mapping = {
    '0-10': 1,
    '11-20': 2,
    '21-30': 3,
    '31-40': 4,
    '41-50': 5,
    '51-60': 6,
    '61-70': 7,
    '71-80': 8,
    '81-90': 9,
    '91-100': 10
}

severity_mapping = {
    'Minor': 1,
    'Moderate': 2,
    'Extreme': 3
}

admission_type_mapping = {
    'Urgent': 1,
    'Emergency': 2,
    'Trauma': 3
}

department_mapping = {
    'gynecology': 1,
    'anesthesia': 2,
    'radiotherapy': 3,
    'TB & Chest disease': 4,
    'surgery': 5
}

hospital_type_mapping = {
    'Clinic': 1,
    'Community Hospital': 2,
    'Public Hospital': 3,
    'General Medical & Surgical Hospital': 4,
    'Specialty Hospital': 5,
    'Acute Hospital': 6,
    'Teaching Hospital': 7
}

In [23]:
df_stay['Stay'] = df_stay['Stay'].map(stay_mapping_1)

df_stay['Age'] = df_stay['Age'].map(age_mapping)
df_stay['Severity of Illness'] = df_stay['Severity of Illness'].map(severity_mapping)
df_stay['Type of Admission'] = df_stay['Type of Admission'].map(admission_type_mapping)
df_stay['Department'] = df_stay['Department'].map(department_mapping)
df_stay['Hospital Type'] = df_stay['Hospital Type'].map(hospital_type_mapping)

In [24]:
df_stay['Stay'].value_counts()

Stay
Standard Stay      87491
Short-term Stay    78139
Moderate Stay      55159
Prolonged Stay     46761
Extended Stay      27284
Brief Stay         23604
Name: count, dtype: int64

In [25]:
# Define a new mapping from category names to integers
stay_mapping_2 = {
    'Brief Stay': 1,
    'Short-term Stay': 2,
    'Standard Stay': 3,
    'Moderate Stay': 4,
    'Prolonged Stay': 5,
    'Extended Stay': 6
}

In [26]:
df_stay['Stay'] = df_stay['Stay'].map(stay_mapping_2)

In [27]:
df_stay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318438 entries, 0 to 318437
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   Stay                 318438 non-null  int64
 1   Age                  318438 non-null  int64
 2   Severity of Illness  318438 non-null  int64
 3   Type of Admission    318438 non-null  int64
 4   Department           318438 non-null  int64
 5   Hospital Type        318438 non-null  int64
dtypes: int64(6)
memory usage: 14.6 MB


In [28]:
df_stay.head()

,Stay,Age,Severity of Illness,Type of Admission,Department,Hospital Type
0,1,6,3,2,3,5
1,5,6,3,3,3,5
2,4,6,3,3,2,1
3,5,6,3,3,3,4
4,5,6,3,3,3,4


In [29]:
df_stay['Stay'].value_counts()

Stay
3    87491
2    78139
4    55159
5    46761
6    27284
1    23604
Name: count, dtype: int64

In [31]:
# Find the minimum class count for undersampling
min_class_count = df_stay['Stay'].value_counts().min()


In [33]:
# Create an empty DataFrame to store the balanced data
df_stay_balanced = pd.DataFrame()

In [35]:
# Loop through each class and sample `min_class_count` instances from each
for stay_class in df_stay['Stay'].unique():
    # Filter the class-specific data
    class_data = df_stay[df_stay['Stay'] == stay_class]
    
    # Perform undersampling
    class_data_resampled = resample(class_data,
                                    replace=False,           # Do not replace, to perform undersampling
                                    n_samples=min_class_count,  # Match the minority class count
                                    random_state=42)         # For reproducibility
    
    # Append the resampled class data to `df_balanced`
    df_stay_balanced = pd.concat([df_stay_balanced, class_data_resampled])

In [36]:
# Shuffle the balanced dataset to ensure randomness
df_stay_balanced = df_stay_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [37]:
df_stay_balanced['Stay'].value_counts()

Stay
2    23604
3    23604
5    23604
6    23604
1    23604
4    23604
Name: count, dtype: int64

In [38]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import joblib

In [39]:
# Define parameter grids for both models
rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0]
}


In [40]:
x = df_stay_balanced.drop(columns='Stay')  # Replace with the correct feature columns
y = df_stay_balanced['Stay']

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

### Random Forest

In [42]:
# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=42)

In [43]:
# Perform Grid Search with cross-validation
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, scoring='accuracy', cv=5, n_jobs=-1)

In [44]:
rf_grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, None], 'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [100, 200]},
             scoring='accuracy')

In [45]:
# Get the best Random Forest model from the Grid Search
best_rf_model = rf_grid_search.best_estimator_
print('Best Parameter for Random Forest:', rf_grid_search.best_estimator_)

Best Parameter for Random Forest: RandomForestClassifier(max_depth=10, n_estimators=200, random_state=42)


### XGBoost

In [46]:
# Initialize XGBoost model
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')

In [47]:
# Perform Grid Search with accuracy as scoring metric
xgb_grid_search = GridSearchCV(xgb_model, xgb_param_grid, scoring='accuracy', cv=5, n_jobs=-1)

In [49]:
# Adjust y_train and y_test to start labels from 0
y_train_adjusted = y_train - 1
y_test_adjusted = y_test - 1

In [50]:
xgb_grid_search.fit(x_train, y_train_adjusted)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='mlogloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': [3, 5, 7],
                         'n_estimators': [100, 200], 'subsample': [0.8, 1.0]},
             scoring='accuracy')

In [51]:
# Get the best XGBoost model from the Grid Search
best_xgb_model = xgb_grid_search.best_estimator_

### Evaluate and Export

In [55]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import joblib

def evaluate_models(model_rf, model_xgb, X_test, y_test, scoring_metric='accuracy'):
    # Predict on the test set
    rf_predictions = model_rf.predict(X_test)
    xgb_predictions = model_xgb.predict(X_test)
    
    # Define metrics
    metrics = {
        'Accuracy': accuracy_score,
        'F1 Score': f1_score,
        'Precision': precision_score,
        'Recall': recall_score
    }
    
    # Function to handle scoring with or without 'average' parameter
    def apply_metric(metric_func, y_true, y_pred):
        if metric_func in [f1_score, precision_score, recall_score]:
            return metric_func(y_true, y_pred, average='weighted')
        else:
            return metric_func(y_true, y_pred)
    
    # Evaluate each model
    rf_results = {metric: apply_metric(metrics[metric], y_test, rf_predictions) for metric in metrics}
    xgb_results = {metric: apply_metric(metrics[metric], y_test, xgb_predictions) for metric in metrics}
    
    # Display results
    print("Random Forest Results:", rf_results)
    print("XGBoost Results:", xgb_results)
    
    # Determine the best model based on the chosen metric
    if rf_results[scoring_metric.capitalize()] > xgb_results[scoring_metric.capitalize()]:
        best_model = model_rf
        model_name = 'Random Forest'
    else:
        best_model = model_xgb
        model_name = 'XGBoost'
    
    print(f"Best model: {model_name} based on {scoring_metric.capitalize()}")
    
    # Save the best model to disk
    joblib.dump(best_model, f'best_model_{model_name}.pkl')
    print(f"Model saved as best_model_{model_name}.pkl")
    
    return best_model, rf_results, xgb_results


In [56]:
best_model, rf_metrics, xgb_metrics = evaluate_models(best_rf_model, best_xgb_model, x_test, y_test, scoring_metric='accuracy')

Random Forest Results: {'Accuracy': 0.2456134157105031, 'F1 Score': 0.2174575352311833, 'Precision': 0.23138048001351372, 'Recall': 0.2456134157105031}
XGBoost Results: {'Accuracy': 0.11152691968225949, 'F1 Score': 0.11669718065283745, 'Precision': 0.14660790145727526, 'Recall': 0.11152691968225949}
Best model: Random Forest based on Accuracy
Model saved as best_model_Random Forest.pkl
